In [2]:
import socket
import threading
from Crypto.Cipher import DES
from Crypto.Util.Padding import unpad


'''DES Encryption - same as client - added up top for clarity'''
des_key = b'password'
cipher = DES.new(des_key, DES.MODE_ECB)

# Server setup
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket object using IPv4 and TCP protocols
server.bind(('localhost', 43210))  # Bind the server to localhost on port 12345
server.listen()  # Listen for incoming connections

clients = []  # List to keep track of clients
nicknames = []  # List to keep track of client nicknames

def broadcast(message):
    """Send a message to all connected clients."""
    for client in clients:
        try:
            client.send(message)
        except:
            # Handle the case where sending message fails
            index = clients.index(client)
            clients.remove(client)
            client.close()
            nickname = nicknames[index]
            broadcast(f'{nickname} left the chat!'.encode('utf-8'))
            nicknames.remove(nickname)

def handle(client):
    """Handle messages from clients."""
    while True:
        try:
            encrypted_message = client.recv(1024)
            if encrypted_message:
                if len(encrypted_message) % DES.block_size == 0:
                    decrypted_message = unpad(cipher.decrypt(encrypted_message), DES.block_size).decode()
                    nickname = nicknames[clients.index(client)]
                    print(f"Encrypted message from {nicknames[clients.index(client)]}: {encrypted_message}")
                    print(f"Unencrypted message from {nicknames[clients.index(client)]}: {decrypted_message}")
                    broadcast(encrypted_message)  
                else:
                    print("Received misaligned data")
            ''' Commented lines below from pre-encryption version
            message = client.recv(1024)  # Try to receive message from client
            broadcast(message)  # Broadcast received message to all clients
            '''
        except Exception as e:
            print(f"An error occurred: {e}")
            break

def receive():
    """Accept new connections and start a new thread for each client."""
    while True:
        try:
            client, address = server.accept()  # Accept a new connection
            print(f"Connected with {str(address)}")

            client.send('NICK'.encode('utf-8'))  # Request a nickname from the client
            nickname = client.recv(1024).decode('utf-8')  # Receive the nickname
            nicknames.append(nickname)  # Add nickname to the list
            clients.append(client)  # Add client to the list

            print(f'Nickname of the client is {nickname}!')
            broadcast(f'{nickname} joined the chat!'.encode('utf-8'))  # Notify all clients about the new client
            client.send('Connected to the server!'.encode('utf-8'))  # Notify the new client of successful connection

            thread = threading.Thread(target=handle, args=(client,))  # Start handling thread for the client
            thread.start()
        except KeyboardInterrupt:
            # Here you can add any cleanup code you need
            print("Server is shutting down...")
            for client in clients:
                client.close()
            server.close()
            break
        except:
            # Handle other exceptions
            break

try:
    print("Server is listening...")
    receive()  # Start accepting connections
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    server.close()    

Server is listening...
Connected with ('127.0.0.1', 58706)
Nickname of the client is jono!
Connected with ('127.0.0.1', 58711)
Nickname of the client is bono!
Connected with ('127.0.0.1', 58778)
Nickname of the client is Bon!
Connected with ('127.0.0.1', 58789)
Nickname of the client is boris!
Connected with ('127.0.0.1', 59286)
Nickname of the client is Dor!
Connected with ('127.0.0.1', 59340)
Nickname of the client is Toomo!
Connected with ('127.0.0.1', 59518)
Nickname of the client is Tommyo!
Connected with ('127.0.0.1', 59522)
Nickname of the client is To!


KeyboardInterrupt: 